# Trabajo Práctico 1: Propiedades en Venta - Gradient Boost

## Grupo 11 - "Los Outliers"
- Castillo, Carlos
- Destefanis, Juan Pablo
- Gómez, Celeste

## Libraries

In [5]:
import pandas as pd
import numpy as np

## Load del DataFrame a utilizar

Cargo los datos con los avisos clasificados con su tipo de precio. También fijamos una semilla para usarla a lo largo de toda la notebook.

In [6]:
df = pd.read_csv("https://drive.google.com/uc?id=1GhsJwy29gS2y_HibaDeChkx-ozSc2Qc3", index_col=0)
semilla = 137

## Limpieza de los datos

Lo primero que hacemos para poder entrenar el modelo de Gradient Boost es seleccionar las columnas del dataframe que consideramos más relevantes. Hay algunas, como el `id` y  `property_title`, que claramente no aportan información útil. También, removemos la columna de `property_price` por ser el target, y las columnas de `precio_m2` y `tipo_precio` por estar íntimamente relacionadas. Finalmente, usamos un *label encoder* para las columnas `neighbourhood` y `property_type`, que necesitan ser numéricas para que las pueda considerar el modelo. Dado que queremos evitar tener muchas columnas, optamos por no usar One Hot Encoding.

In [7]:
from sklearn.preprocessing import LabelEncoder

df_features = df[[
    "latitud",
    "longitud",
    "neighbourhood",
    "property_type",
    "property_rooms",
    "property_bedrooms",
    "property_surface_total",
    "property_surface_covered"]]

df_target = df[["property_price"]]

encoder = LabelEncoder()
df_features.loc[:, 'property_type'] = encoder.fit_transform(df_features.property_type.values)
df_features.loc[:, 'neighbourhood'] = encoder.fit_transform(df_features.neighbourhood.values)

Luego, genero los conjuntos de train y test.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target.values.ravel(), test_size=0.20, random_state=semilla)

## Entrenamiento de un primer modelo

Empezamos entrenando un modelo base, para luego entrenar otro optimizando sus hiperparámetros y poder compararlos.

In [14]:
from sklearn import ensemble

regressor = ensemble.GradientBoostingRegressor(random_state=semilla)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(random_state=137)

Observo las métricas del modelo en testing y en training.

In [16]:
from sklearn.metrics import mean_squared_error, r2_score

prediccion_train = regressor.predict(X_train)
mse = mean_squared_error(y_train, prediccion_train)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, prediccion_train)

print("\nMétricas con el conjunto de training.\n")
print("RMSE (train): {:.4f}".format(rmse))
print("MSE (train): {:.4f}".format(mse))
print("R2 (train): {:.4f}".format(r2))

prediccion_test = regressor.predict(X_test)
mse = mean_squared_error(y_test, prediccion_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, prediccion_test)

print("\nMétricas con el conjunto de testing.\n")
print("RMSE (test): {:.4f}".format(rmse))
print("MSE (test): {:.4f}".format(mse))
print("R2 (test): {:.4f}".format(r2))


Métricas con el conjunto de training.

RMSE (train): 58088.2673
MSE (train): 3374246792.6480
R2 (train): 0.8096

Métricas con el conjunto de testing.

RMSE (test): 59288.4529
MSE (test): 3515120646.7748
R2 (test): 0.8008


## Randomized Search

Una vez entrenado un modelo base para poder comparar luego, buscamos entrenar un nuevo modelo optimizando los siguientes hiperparámetros:

* `n_stimators`: Cantidad de árboles máximos que pueden estar presentes en el modelo.
* `max_depth`: Máximo nivel de profundidad que puede tener cada árbol.
* `learning_rate`: Valor utilizado para disminuir que tanto contribuye cada árbol al resultado final.
* `loss`: Función de perdida a optimizar. Vamos a probar con el error cuadrático, el error absoluto y "huber", que es una combinación de ambos.


In [19]:
params_grid = {
    "n_estimators": range(20, 50),
    "max_depth": [8, 16, 24, 32],
    "learning_rate": [0.01, 0.1, 0.2],
    "loss": ["squared_error", "absolute_error"],
}

De manera similar a como lo hicimos en la parte de clasificación, usamos Stratified KFold porque hay una distribución dispareja en las categorías del target que queremos tratar de mantener en cada fold. También, elegimos usar 5 folds.

Por otro lado, y también como lo hicimos en la etapa de clasificación, utilizamos RandomizedSearch en lugar de GridSearch para ahorrar tiempo mientras experimentamos.

In [20]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

folds=5
kfoldcv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=semilla)

base_regressor = ensemble.GradientBoostingRegressor(random_state=semilla)

randomcv = RandomizedSearchCV(
    estimator=base_regressor,
    scoring="r2",
    param_distributions=params_grid,
    n_iter=10,
    cv=kfoldcv,
    random_state=semilla,
    verbose=2
)

Para entrenar al modelo utilizamos el método fit con el set de entrenamiento ya transformado:

```python
randomcv.fit(X_train, y_train)
```

Sin embargo, luego de haber ejecutado este método con anterioridad, ya contamos con el modelo más óptimo encontrado, que ha sido exportado en un archivo de joblib, lo que nos permite simplemente cargar el archivo y no tener que volver a entrenar todos los modelos con todos los parámetros que probamos.

In [35]:
from joblib import load

best_regressor = load("gradient_boost.joblib")
best_regressor.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.2,
 'loss': 'absolute_error',
 'max_depth': 32,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 41,
 'n_iter_no_change': None,
 'random_state': 137,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

Finalmente observamos las estadísticas del modelo.

In [34]:
prediccion_train = best_regressor.predict(X_train)
mse = mean_squared_error(y_train, prediccion_train)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, prediccion_train)

print("\nMétricas con el conjunto de training.\n")
print("RMSE (train): {:.4f}".format(rmse))
print("MSE (train): {:.4f}".format(mse))
print("R2 (train): {:.4f}".format(r2))

prediccion_test = best_regressor.predict(X_test)
mse = mean_squared_error(y_test, prediccion_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, prediccion_test)

print("\nMétricas con el conjunto de testing.\n")
print("RMSE (test): {:.4f}".format(rmse))
print("MSE (test): {:.4f}".format(mse))
print("R2 (test): {:.4f}".format(r2))


Métricas con el conjunto de training.

RMSE (train): 16285.7942
MSE (train): 265227094.1160
R2 (train): 0.9850

Métricas con el conjunto de testing.

RMSE (test): 43400.0987
MSE (test): 1883568568.2602
R2 (test): 0.8933


Si comparamos el rendimiento de este modelo con el que utilizaba los parámetros por defecto, notamos que aunque el anterior ya tenía un rendimiento significativamente positivo, este lo mejora aún más. Por el resultado de las métricas en el conjunto de training, especialmente $R^2$, podríamos sospechar que el modelo tiene overfitting, sin embargo, también muestra un desempeño robusto en el conjunto de test ante datos que nunca ha visto.